<a href="https://colab.research.google.com/github/felipednegredo/tcc-emocoes-musicais-codigo/blob/main/TCC_Carregamento_dos_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Carregamento das depedências

In [ ]:
from __future__ import annotations
from pathlib import Path
from dataclasses import dataclass
from typing import List, Dict, Any, Optional, Tuple
from collections import defaultdict, Counter
import pandas as pd
import numpy as np
import re
import json
import warnings
import os
import sys
from datetime import datetime

## Config

In [ ]:
base = Path("/content/drive/MyDrive/DataSet TCC/DEAM")
BASE_DIR = base
META_FILES = [
    base / "metadata" / "metadata_2013.csv",
    base / "metadata" / "metadata_2014.csv",
    base / "metadata" / "metadata_2015.csv",
]
AUDIO_EXTS = {".wav", ".mp3", ".flac", ".m4a", ".ogg"}
TIPOS_RELEVANTES = {"audio", "features"}
DYN_VAL_DIR = base / "annotations per each rater" / "dynamic (per second annotations)" / "valence"
DYN_ARO_DIR = base / "annotations per each rater" / "dynamic (per second annotations)" / "arousal"
STATIC_ANN_FILE = base / "metadata" / "static_annotations_songs_1_2000.csv"
COL_CANON = {
    "id": {"id","track_id","song_id","music_id","ID"},
    "filename": {"filename","file","file_name"},
    "title": {"title","song title","song_title","track","name"},
    "artist": {"artist","artists","author"},
    "album": {"album","record"},
    "genre": {"genre","style"},
    "start": {"start","start of the segment","start of the segment (min.sec)","start (min.sec)","start_sec"},
    "end": {"end","end of the segment","end of the segment (min.sec)","end (min.sec)","end_sec"},
}

## Utils

In [ ]:
# ===============
# Utils & Logging
# ===============
def log(msg: str):
    print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}")

def _strip_text(s):
    if pd.isna(s):
        return s
    return re.sub(r"\s+", " ", str(s)).strip()

def _normalize_cols(df: pd.DataFrame, col_canon: Dict[str, set]) -> pd.DataFrame:
    norm = {c: re.sub(r"\s+", " ", str(c).strip().lower()) for c in df.columns}
    df = df.rename(columns=norm)
    rename_map = {}
    for canon, variants in col_canon.items():
        lower_variants = {v.lower() for v in variants}
        for c in list(df.columns):
            if c in lower_variants:
                rename_map[c] = canon
    df = df.rename(columns=rename_map)
    return df

def minsec_to_seconds(v):
    if pd.isna(v):
        return np.nan
    s = str(v).strip()
    if s == "":
        return np.nan
    if ":" in s:
        try:
            mm, ss = s.split(":")
            return int(mm) * 60 + float(ss)
        except Exception:
            return np.nan
    if "." in s:
        try:
            mm, ss = s.split(".")
            return int(mm) * 60 + float(ss)
        except Exception:
            return np.nan
    try:
        return float(s)
    except Exception:
        return np.nan

def _extract_id_from_filename(fname: str) -> Optional[str]:
    if not isinstance(fname, str):
        fname = str(fname)
    m = re.search(r'(?<!\d)(\d+)(?!\d)', fname)
    if m:
        num = m.group(1).lstrip("0")
        return num or "0"
    return None

def _coerce_id_col(df: pd.DataFrame) -> pd.DataFrame:
    if "id" in df.columns:
        df["id"] = (df["id"].astype(str)
                    .str.extract(r'(\d+)', expand=False)
                    .fillna("")
                    .apply(lambda s: (s.lstrip("0") or "0") if s else ""))
    if ("id" not in df.columns) or (df["id"].eq("").all()):
        if "filename" in df.columns:
            df["id"] = df["filename"].apply(lambda s: _extract_id_from_filename(s) or "")
        else:
            df["id"] = ""
    df = df[df["id"].str.len() > 0].copy()
    return df

def _year_priority(name: str) -> int:
    # 2015 > 2014 > 2013
    if "2015" in name: return 3
    if "2014" in name: return 2
    if "2013" in name: return 1
    return 0

def _row_quality_score(r: pd.Series) -> int:
    score = 0
    if pd.notna(r.get("filename")) and str(r.get("filename")).strip() not in ("", "—", "NaN"):
        score += 3
    has_start_end = (pd.notna(r.get("start_sec")) and pd.notna(r.get("end_sec")))
    if has_start_end:
        score += 3
    if pd.notna(r.get("title")) and str(r.get("title")).strip():
        score += 1
    if pd.notna(r.get("artist")) and str(r.get("artist")).strip():
        score += 1
    return score

def read_csv_resilient(path: Path) -> Optional[pd.DataFrame]:
    if not path.exists():
        log(f"⚠️ CSV não encontrado: {path}")
        return None
    # tenta alguns padrões comuns
    trials = [
        {"sep": ",", "engine": "python"},
        {"sep": None, "engine": "python"},  # deixa inferir
        {"sep": ";", "engine": "python"},
        {"sep": "\t", "engine": "python"},
    ]
    for t in trials:
        try:
            df = pd.read_csv(path, **t)
            return df
        except Exception:
            continue
    log(f"⚠️ Falha ao ler CSV: {path.name}")
    return None

# Carregamento das informações

### Descoberta dos arquivos

In [ ]:
# def index_files(cfg: DeamConfig) -> pd.DataFrame: # Old signature
def index_files(base_dir: Path, tipos_relevantes: set, audio_exts: set) -> pd.DataFrame: # New signature
    arquivos_agrupados: Dict[str, Dict[str, Path]] = defaultdict(dict)
    # if not cfg.BASE_DIR.exists(): # Old
    if not base_dir.exists(): # New
        # log(f"⚠️ BASE_DIR não encontrado: {cfg.BASE_DIR}") # Old
        log(f"⚠️ BASE_DIR não encontrado: {base_dir}") # New
        return pd.DataFrame(columns=["ID","audio","features"])

    # for p in cfg.BASE_DIR.rglob("*"): # Old
    for p in base_dir.rglob("*"): # New
        if not p.is_file():
            continue
        tipo = p.parent.name  # "audio" / "features" etc.
        # if tipo not in cfg.TIPOS_RELEVANTES: # Old
        if tipo not in tipos_relevantes: # New
            continue
        file_id = p.stem
        if not file_id.isdigit():
            # Às vezes, arquivos em "audio" têm nomes com prefixos; tente extrair ID numérico
            extracted = _extract_id_from_filename(p.name)
            if extracted is None:
                continue
            file_id = extracted
        # if tipo == "audio" and p.suffix.lower() not in cfg.AUDIO_EXTS: # Old
        if tipo == "audio" and p.suffix.lower() not in audio_exts: # New
            continue
        arquivos_agrupados[file_id][tipo] = p

    rows: List[Dict[str, Any]] = []
    for file_id, tipos in arquivos_agrupados.items():
        row = {"ID": file_id}
        for tipo, path in tipos.items():
            try:
                # row[tipo] = str(path.relative_to(cfg.BASE_DIR)) # Old
                row[tipo] = str(path.relative_to(base_dir)) # New
            except Exception:
                row[tipo] = str(path)
        rows.append(row)

    df_ids = pd.DataFrame(rows).fillna("—")
    if not df_ids.empty:
        df_ids = df_ids.sort_values(by="ID", key=lambda s: s.astype(int))
    log(f"✅ IDs encontrados na árvore: {len(df_ids)} (com audio/features)")
    return df_ids

## Utils

In [ ]:
def _infer_year_from_source(name: str) -> int:
    if "2015" in name: return 2015
    if "2014" in name: return 2014
    if "2013" in name: return 2013
    return 0

def _harmonize_filename(s: Any) -> Any:
    """Normaliza filename para comparação estável: remove diretórios, normaliza separadores e espaços."""
    if pd.isna(s):
        return s
    s = str(s).strip().replace("\\", "/")
    base = s.split("/")[-1]
    return re.sub(r"\s+", " ", base).strip()

def _sanitize_segments(df: pd.DataFrame) -> pd.DataFrame:
    """Garante start_sec/end_sec coerentes, tenta converter de start/end em texto, troca quando end<start e cria segment_duration_s."""
    # Converte se vieram como texto
    if "start_sec" in df.columns and not np.issubdtype(df["start_sec"].dtype, np.number):
        df["start_sec"] = df["start_sec"].apply(minsec_to_seconds)
    if "end_sec" in df.columns and not np.issubdtype(df["end_sec"].dtype, np.number):
        df["end_sec"] = df["end_sec"].apply(minsec_to_seconds)

    # Deriva de start/end (m.ss ou mm:ss)
    need_start = df.get("start_sec").isna() & df.get("start").notna() if "start_sec" in df and "start" in df else pd.Series(False, index=df.index)
    if need_start.any():
        df.loc[need_start, "start_sec"] = df.loc[need_start, "start"].apply(minsec_to_seconds)

    need_end = df.get("end_sec").isna() & df.get("end").notna() if "end_sec" in df and "end" in df else pd.Series(False, index=df.index)
    if need_end.any():
        df.loc[need_end, "end_sec"] = df.loc[need_end, "end"].apply(minsec_to_seconds)

    # Se end < start, troca
    if "start_sec" in df.columns and "end_sec" in df.columns:
        mask_swap = df["start_sec"].notna() & df["end_sec"].notna() & (df["end_sec"] < df["start_sec"])
        if mask_swap.any():
            s_old = df.loc[mask_swap, "start_sec"].copy()
            df.loc[mask_swap, "start_sec"] = df.loc[mask_swap, "end_sec"]
            df.loc[mask_swap, "end_sec"] = s_old

        # Duração
        df["segment_duration_s"] = df.apply(
            lambda r: (float(r["end_sec"]) - float(r["start_sec"]))
                      if pd.notna(r["end_sec"]) and pd.notna(r["start_sec"]) else np.nan,
            axis=1
        )
        # Elimina negativos residuais
        df.loc[df["segment_duration_s"] < 0, "segment_duration_s"] = np.nan
    else:
        df["segment_duration_s"] = np.nan

    return df

## Carregamento dos dados

In [ ]:
# def load_all_metadata(cfg: DeamConfig) -> pd.DataFrame: # Old signature
def load_all_metadata(meta_files: List[Path], col_canon: Dict[str, set]) -> pd.DataFrame: # New signature
    frames = []
    # for csvp in cfg.META_FILES: # Old
    for csvp in meta_files: # New
        df_m = read_csv_resilient(csvp)
        if df_m is None:
            continue

        # Remove colunas lixo comuns (ex.: Unnamed: 0)
        drop_garbage = [c for c in df_m.columns if str(c).lower().startswith("unnamed")]
        if drop_garbage:
            df_m = df_m.drop(columns=drop_garbage, errors="ignore")

        # Normaliza nomes (canônicos + aliases extras típicos do DEAM)
        # df_m = _normalize_cols(df_m, cfg.COL_CANON) # Old
        df_m = _normalize_cols(df_m, col_canon) # New

        # Aliases adicionais frequentes
        alias_map = {}
        for c in list(df_m.columns):
            cl = str(c).strip().lower()
            if cl in {"performer","singer","band"}: alias_map[c] = "artist"
            if cl in {"track name","songname","song"}: alias_map[c] = "title"
            if cl in {"begin","start time","segment start"}: alias_map[c] = "start"
            if cl in {"end time","segment end"}: alias_map[c] = "end"
            if cl in {"file path","path"}: alias_map[c] = "filename"
        if alias_map:
            df_m = df_m.rename(columns=alias_map)

        # Garante colunas esperadas
        expected = ["id","filename","title","artist","album","genre","start","end","start_sec","end_sec"]
        for k in expected:
            if k not in df_m.columns:
                df_m[k] = pd.NA

        # Limpeza de texto
        for col in ["filename","title","artist","album","genre","start","end"]:
            if col in df_m.columns:
                df_m[col] = df_m[col].apply(_strip_text)

        # Harmoniza filename (basename somente)
        df_m["filename"] = df_m["filename"].apply(_harmonize_filename)

        # ID canônico (a partir de id ou extraído do filename)
        df_m = _coerce_id_col(df_m)

        # Saneia segmentos (conversões, troca se end<start, cria duration)
        df_m = _sanitize_segments(df_m)

        # Metadado de origem
        df_m["source_file"] = csvp.name
        df_m["_year"] = _infer_year_from_source(csvp.name)

        # Heurística de qualidade
        df_m["quality_score"] = df_m.apply(_row_quality_score, axis=1)

        # Drop linhas totalmente vazias de interesse (sem id válido já foi filtrado; aqui tiramos as sem nada útil)
        key_useful = df_m[["filename","title","artist","start_sec","end_sec"]].notna().any(axis=1)
        df_m = df_m[key_useful].copy()

        frames.append(df_m)

    if not frames:
        return pd.DataFrame(columns=[
            "ID","filename","title","artist","album","genre",
            "start","end","start_sec","end_sec","segment_duration_s",
            "source_file"
        ])

    meta_all = pd.concat(frames, ignore_index=True, sort=False)

    # ---- Resolução de conflitos por ID ----
    # Critérios (em ordem):
    # 1) quality_score (desc)
    # 2) _year (desc) → 2015 > 2014 > 2013
    # 3) segment_duration_s (desc) — se houver recortes, preferir o mais longo
    # 4) first occurrence (estável)
    sort_cols = ["id", "quality_score", "_year", "segment_duration_s"]
    for sc in sort_cols:
        if sc not in meta_all.columns:
            meta_all[sc] = np.nan if sc != "id" else meta_all.get("id", "")

    meta_all = (meta_all
                .sort_values(sort_cols, ascending=[True, False, False, False], kind="mergesort")
                .drop_duplicates(subset=["id"], keep="first"))

    # Renomeia id → ID e organiza colunas
    meta_all = meta_all.rename(columns={"id":"ID"})
    col_order = [
        "ID","filename","title","artist","album","genre",
        "start","end","start_sec","end_sec","segment_duration_s",
        "source_file"
    ]
    for c in col_order:
        if c not in meta_all.columns:
            meta_all[c] = pd.NA
    meta_all = meta_all[col_order].copy()

    # Ordena por ID numérico quando possível
    meta_all = meta_all.sort_values(by="ID", key=lambda s: s.astype(str).str.extract(r'(\d+)', expand=False).fillna("0").astype(int))

    log(f"✅ IDs únicos nos metadados: {meta_all['ID'].nunique()}")
    # Diagnóstico rápido
    n_bad_seg = meta_all["segment_duration_s"].isna().sum()
    if n_bad_seg:
        log(f"ℹ️  Aviso: {n_bad_seg} linhas sem duração de segmento (falta start/end válidos).")

    return meta_all

In [ ]:
def _clean_colname(c: str) -> str:
    c = str(c)
    c = c.strip().lower()
    c = re.sub(r"\s+", "_", c)
    c = re.sub(r"[^\w:._\-]+", "", c)  # remove símbolos estranhos
    # evita nomes começando por número
    if re.match(r"^\d", c):
        c = "_" + c
    return c

def _read_features_file(path: Path) -> pd.DataFrame | None:
    """
    Le com fallback: parquet -> csv -> tsv
    Normaliza colunas. Não assume esquema fixo.
    """
    try:
        if path.suffix.lower() == ".parquet":
            df = pd.read_parquet(path)
        elif path.suffix.lower() == ".csv":
            # tentativas de separador
            for sep in [",", None, ";", "\t"]:
                try:
                    df = pd.read_csv(path, sep=sep, engine="python")
                    break
                except Exception:
                    df = None
            if df is None:
                return None
        elif path.suffix.lower() == ".tsv":
            df = pd.read_csv(path, sep="\t", engine="python")
        else:
            return None
    except Exception:
        return None

    if df is None or df.empty:
        return None

    # normaliza nomes
    df = df.rename(columns={c: _clean_colname(c) for c in df.columns})

    # remove colunas “Unnamed”
    drop_garbage = [c for c in df.columns if c.startswith("unnamed")]
    if drop_garbage:
        df = df.drop(columns=drop_garbage, errors="ignore")

    # tenta converter tudo que é numérico
    for c in df.columns:
        if c == "time" or c.endswith("_time") or c.endswith(":time"):
            df[c] = pd.to_numeric(df[c], errors="coerce")
        else:
            # tentativa leve; se virar tudo NaN, mantém string
            s = pd.to_numeric(df[c], errors="coerce")
            # heurística: se mais de 50% vira número, usa numérico
            if s.notna().mean() >= 0.5:
                df[c] = s

    # remove linhas vazias de numéricos (em arquivos “por frame” com sujeira)
    if df.select_dtypes(include=[np.number]).shape[1] > 0:
        allnan_rows = df.select_dtypes(include=[np.number]).isna().all(axis=1)
        if allnan_rows.any():
            df = df.loc[~allnan_rows].copy()

    return df if not df.empty else None

In [ ]:
def _is_per_frame(df: pd.DataFrame) -> bool:
    # Heurística: tem coluna 'time' ou muitas linhas >> colunas
    has_time = "time" in df.columns or any(c.endswith("_time") for c in df.columns)
    many_rows = len(df) > 200 and df.shape[0] > df.shape[1]  # mais linhas que colunas
    return bool(has_time or many_rows)

def _aggregate_features(df: pd.DataFrame, agg_prefix: str = "") -> pd.DataFrame:
    """
    Agrega por coluna numérica: mean, std, min, p25, p50, p75, max.
    Retorna 1 linha com colunas prefixadas.
    """
    num = df.select_dtypes(include=[np.number])
    if num.empty:
        # Se não tem numéricos, tenta contar categorias (raro em features)
        return pd.DataFrame({f"{agg_prefix}count_rows": [len(df)]})

    stats = {
        "mean": num.mean(axis=0),
        "std":  num.std(axis=0, ddof=0),
        "min":  num.min(axis=0),
        "p25":  num.quantile(0.25, axis=0),
        "p50":  num.quantile(0.50, axis=0),
        "p75":  num.quantile(0.75, axis=0),
        "max":  num.max(axis=0),
    }
    out = {}
    for stat_name, s in stats.items():
        for col, val in s.items():
            out[f"{agg_prefix}{col}__{stat_name}"] = float(val) if pd.notna(val) else np.nan
    return pd.DataFrame([out]).astype("Float32")


In [ ]:
def load_all_features(df_tree: pd.DataFrame, base_dir: Path) -> pd.DataFrame:
    """
    Varre df_tree['features'] e carrega/agrupa todas as features por ID.
    Se houver múltiplos arquivos por ID, agrega cada um e concatena por colunas com prefixos.
    Retorna DataFrame: ID + <features...>
    """
    if "ID" not in df_tree.columns or "features" not in df_tree.columns:
        return pd.DataFrame(columns=["ID"])

    rows = []
    for _, r in df_tree.iterrows():
        _id = str(r["ID"])
        feat_rel = r.get("features")
        if pd.isna(feat_rel) or str(feat_rel) == "—":
            continue

        # pode ser único arquivo ou subpasta; vamos tentar resolver ambos
        feat_path = base_dir / str(feat_rel)
        files = []
        if feat_path.is_file() and feat_path.suffix.lower() in FEATURES_EXTS:
            files = [feat_path]
        elif feat_path.is_dir():
            files = [p for p in feat_path.rglob("*") if p.suffix.lower() in FEATURES_EXTS]
        else:
            # pode ter sido caminho absoluto gravado; tenta como Path puro
            p2 = Path(str(feat_rel))
            if p2.exists() and p2.suffix.lower() in FEATURES_EXTS:
                files = [p2]

        if not files:
            continue

        per_file_aggs = []
        for f in sorted(files):
            df_f = _read_features_file(f)
            if df_f is None:
                continue

            # Decide se é por-frame e agrega
            prefix = f.stem  # basename sem extensão
            prefix = _clean_colname(prefix) + "__"

            if _is_per_frame(df_f):
                agg = _aggregate_features(df_f, agg_prefix=prefix)
            else:
                # já é por-faixa (summary): apenas garante numéricos e prefixa
                num = df_f.select_dtypes(include=[np.number])
                if num.empty:
                    continue
                # Se tiver mais de 1 linha, agrega do mesmo jeito
                agg = _aggregate_features(df_f, agg_prefix=prefix) if len(df_f) > 1 else num.astype("Float32")
                # renomeia colunas
                agg = agg.rename(columns=lambda c: f"{prefix}{c}" if not str(c).startswith(prefix) else str(c))
                # garanta 1 linha
                if len(agg) > 1:
                    agg = pd.DataFrame([agg.mean(numeric_only=True)])

            # guarda
            per_file_aggs.append(agg)

        if not per_file_aggs:
            continue

        # concatena as agregações desse ID
        feat_row = pd.concat(per_file_aggs, axis=1)
        # adiciona ID
        feat_row.insert(0, "ID", _id)
        rows.append(feat_row)

    if not rows:
        return pd.DataFrame(columns=["ID"])

    # normaliza: union de colunas
    feats_all = pd.concat(rows, axis=0, ignore_index=True).fillna(np.nan)
    # se mesmo ID apareceu mais de uma vez (múltiplas “pastas” de features),
    # consolidar por média (mantendo 1 linha por ID)
    feats_all = (feats_all
                 .groupby("ID", as_index=False)
                 .mean(numeric_only=True))

    # dtypes enxutos
    for c in feats_all.columns:
        if c == "ID":
            feats_all[c] = feats_all[c].astype(str)
        else:
            feats_all[c] = pd.to_numeric(feats_all[c], errors="coerce").astype("Float32")

    return feats_all

In [ ]:
from __future__ import annotations
from pathlib import Path
from typing import Optional, Iterable, Union, List
import pandas as pd
import numpy as np
import re

# ========= helpers =========

def _detect_delimiter(path: Union[str, Path], default: str = ",") -> str:
    """Detecta ; ou , olhando a primeira linha não vazia."""
    try:
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                s = line.strip()
                if s:
                    semi = s.count(";")
                    comma = s.count(",")
                    if semi > comma:
                        return ";"
                    if comma > semi:
                        return ","
                    break
    except UnicodeDecodeError:
        with open(path, "r", encoding="latin-1") as f:
            for line in f:
                s = line.strip()
                if s:
                    semi = s.count(";")
                    comma = s.count(",")
                    if semi > comma:
                        return ";"
                    if comma > semi:
                        return ","
                    break
    return default

def _read_csv_resilient(path: Union[str, Path], sep: Optional[str] = None) -> Optional[pd.DataFrame]:
    """
    Lê CSV com tolerância: tenta utf-8, cai para latin-1. Auto-detecta separador se não informado.
    """
    path = Path(path)
    if not path.exists() or path.stat().st_size == 0:
        print(f"⚠️  Arquivo ausente ou vazio: {path}")
        return None

    if sep is None:
        sep = _detect_delimiter(path)

    try:
        return pd.read_csv(path, sep=sep, engine="python", on_bad_lines="skip", encoding="utf-8")
    except UnicodeDecodeError:
        try:
            return pd.read_csv(path, sep=sep, engine="python", on_bad_lines="skip", encoding="latin-1")
        except Exception as e:
            print(f"⚠️  Falha ao ler CSV {path}: {e}")
            return None
    except Exception as e:
        print(f"⚠️  Falha ao ler CSV {path}: {e}")
        return None

def _strip_text(x):
    if isinstance(x, str):
        x = x.strip()
        return x if x else pd.NA
    return x

def _maybe_to_numeric_cols(df: pd.DataFrame, exclude: Optional[List[str]] = None) -> pd.DataFrame:
    """
    Converte colunas para numérico quando possível, exceto as listadas em exclude.
    """
    exclude = set(exclude or [])
    for c in df.columns:
        if c in exclude:
            continue
        if pd.api.types.is_numeric_dtype(df[c]):
            continue
        if pd.api.types.is_object_dtype(df[c]):
            df[c] = pd.to_numeric(df[c], errors="ignore")
    return df

# ID: agora pega 1+ dígitos e ignora números entre parênteses
_ID_DIGITS = re.compile(r"\d+")

def parse_id_from_filename(path: Union[str, Path]) -> Optional[int]:
    """
    Extrai o ID principal do nome do arquivo.
    Regras:
      - Remove conteúdos entre parênteses: '2058 (4)' -> '2058 '
      - Busca TODAS as sequências de dígitos e escolhe a MAIS LONGA (ex.: '2058' vs '4')
      - Funciona para '2.csv', '10.csv', '2058 (4).csv', 'track_1732.csv'
    """
    stem = Path(path).stem
    # remove blocos entre parênteses
    stem = re.sub(r"\([^)]*\)", " ", stem)
    # pega todas as sequências de dígitos
    nums = _ID_DIGITS.findall(stem)
    if not nums:
        return None
    # escolhe a sequência mais longa; em empate, a primeira
    nums.sort(key=lambda s: (-len(s), stem.find(s)))
    try:
        return int(nums[0])
    except Exception:
        return None

# ========= loaders =========

def load_feature_single_csv_by_id(
    path: Union[str, Path],
    *,
    id_col_candidates: Iterable[str] = ("ID", "id", "track_id", "song_id"),
    ensure_numeric: bool = True,
    add_source_file: bool = True,
) -> pd.DataFrame:
    """
    Lê UM CSV de features por música.
    - Detecta ;/,, cria/normaliza coluna 'ID' (int) a partir do conteúdo ou do nome do arquivo.
    - Converte colunas numéricas quando possível.
    - Adiciona 'source_file'.
    """
    df = _read_csv_resilient(path)
    if df is None or df.empty:
        print(f"⚠️  Features vazias/ilegíveis: {path}")
        return pd.DataFrame()

    df = df.copy()

    # Descobre ID no conteúdo
    id_series = None
    for c in id_col_candidates:
        if c in df.columns:
            id_series = df[c]
            break

    if id_series is None:
        # ID não existe no conteúdo -> cria a partir do arquivo
        file_id = parse_id_from_filename(path)
        if file_id is None:
            print(f"⚠️  Não foi possível inferir ID a partir do arquivo: {path}")
            df["ID"] = pd.NA
        else:
            df["ID"] = file_id
    else:
        # normaliza para 'ID'
        if id_series.name != "ID":
            df["ID"] = id_series
        # remove colunas duplicadas de id
        for c in id_col_candidates:
            if c in df.columns and c != "ID":
                df = df.drop(columns=[c])

    # Padroniza ID como inteiro quando der
    df["ID"] = pd.to_numeric(df["ID"], errors="coerce").astype("Int64")

    # Strip em colunas texto óbvias
    for cand in ["filename", "path", "file", "track_path", "track"]:
        if cand in df.columns:
            df[cand] = df[cand].apply(_strip_text)

    # Converte numéricos (deixa 'ID' e textos de fora)
    if ensure_numeric:
        df = _maybe_to_numeric_cols(df, exclude=["ID","filename","path","file","track_path","track"])

    if add_source_file and "source_file" not in df.columns:
        df["source_file"] = Path(path).name

    return df

def load_feature_dir_by_id(
    folder: Union[str, Path],
    *,
    glob: str = "*.csv",
    union_columns: bool = True
) -> pd.DataFrame:
    """
    Lê TODOS os CSVs de uma pasta (um CSV por música/ID) e empilha em um único DataFrame.
    - União de colunas entre arquivos (outer concat).
    """
    folder = Path(folder)
    files = sorted(folder.glob(glob))
    if not files:
        print(f"⚠️  Nenhum arquivo encontrado em {folder} com padrão {glob}")
        return pd.DataFrame()

    dfs = []
    for f in files:
        df_i = load_feature_single_csv_by_id(f)
        if df_i is None or df_i.empty:
            continue
        dfs.append(df_i)

    if not dfs:
        return pd.DataFrame()

    df_all = pd.concat(dfs, ignore_index=True, sort=union_columns)
    return df_all

# ========= salvar (opcional) =========

def save_parquet(df: pd.DataFrame, out_path: Union[str, Path], engine: str = "pyarrow") -> None:
    out_path = Path(out_path)
    try:
        df.to_parquet(out_path, engine=engine, index=False)
    except Exception:
        fallback = "fastparquet" if engine == "pyarrow" else "pyarrow"
        df.to_parquet(out_path, engine=fallback, index=False)
    print(f"💾 Salvo em: {out_path.resolve()}")



# Pasta onde estão os CSVs por música (ex.: 2.csv, 10.csv, 2058 (4).csv, ...)
feat_dir = "/content/drive/MyDrive/DataSet TCC/DEAM/features"

# Carrega tudo em um único DF
df_feats = load_feature_dir_by_id(feat_dir, glob="*.csv")
print(df_feats.shape)
df_feats.head()



(457622, 263)


,F0final_sma_amean,F0final_sma_de_amean,F0final_sma_de_stddev,F0final_sma_stddev,ID,audSpec_Rfilt_sma[0]_amean,audSpec_Rfilt_sma[0]_stddev,audSpec_Rfilt_sma[10]_amean,audSpec_Rfilt_sma[10]_stddev,audSpec_Rfilt_sma[11]_amean,...,pcm_zcr_sma_stddev,shimmerLocal_sma_amean,shimmerLocal_sma_de_amean,shimmerLocal_sma_de_stddev,shimmerLocal_sma_stddev,source_file,voicingFinalUnclipped_sma_amean,voicingFinalUnclipped_sma_de_amean,voicingFinalUnclipped_sma_de_stddev,voicingFinalUnclipped_sma_stddev
0,104.9874,0.012754,0.025899,97.24670,10,12.305450,44.153610,43.737370,133.183300,29.232560,...,0.478555,0.057470,-1.262267e-04,0.000700,0.098024,10.csv,0.476549,1.740907e-05,0.000067,0.431432
1,214.8372,0.007899,0.036804,40.45958,10,12.144500,44.169510,42.540280,133.156900,28.281260,...,0.006558,0.118911,-1.191096e-04,0.000708,0.091223,10.csv,0.861955,3.866169e-06,0.000060,0.039230
2,225.7485,0.000403,0.032558,36.87964,10,0.210812,0.234420,0.998713,0.288130,1.085313,...,0.008942,0.123805,2.211471e-06,0.000104,0.088866,10.csv,0.843155,-2.659615e-06,0.000039,0.038720
3,220.6725,0.004453,0.020717,19.60209,10,0.200062,0.214733,1.004283,0.446411,1.092845,...,0.010762,0.107406,-3.993654e-06,0.000053,0.080132,10.csv,0.826222,1.841828e-06,0.000014,0.026931
4,213.4185,-0.000657,0.002777,16.58789,10,0.147441,0.087829,1.371579,0.813228,1.309373,...,0.011198,0.090964,-8.912483e-07,0.000034,0.069354,10.csv,0.833982,6.527346e-08,0.000015,0.034829


## Execução completa

In [ ]:
def build_master(base_dir: Path, meta_files: List[Path], static_ann_file: Optional[Path], dyn_val_dir: Optional[Path], dyn_aro_dir: Optional[Path]) -> Tuple[pd.DataFrame, Dict[str, Any]]:
    # Pass configuration variables as arguments instead of using a CFG object
    df_tree = index_files(base_dir, TIPOS_RELEVANTES, AUDIO_EXTS) # Use the original index_files function

    # load_all_metadata needs META_FILES and COL_CANON. COL_CANON is still a global variable.
    meta_all = load_all_metadata(META_FILES, COL_CANON) # Use the original load_all_metadata function

    # Juntar árvore + metadados
    master = pd.merge(df_tree, meta_all, on="ID", how="outer", validate="one_to_one", suffixes=("",""))

    # Remove unwanted columns that might have been introduced during processing
    cols_to_drop = ['static_ann_file', 'dyn_val_dir', 'dyn_aro_dir']
    master = master.drop(columns=[c for c in cols_to_drop if c in master.columns], errors='ignore')


    # Tipos e limpeza final
    intable_cols = ["ID"]
    for c in master.columns:
        if c == "ID":
            master[c] = master[c].astype(str).str.extract(r'(\d+)', expand=False).fillna("0")
    # ordena
    if not master.empty:
        master = master.sort_values(by="ID", key=lambda s: s.astype(int)).reset_index(drop=True)

    # Relatório de cobertura
    report = {
        "n_ids_tree": int(len(df_tree)),
        "n_ids_meta": int(meta_all["ID"].nunique()),
        "n_audio_paths": int(master["audio"].ne("—").sum()) if "audio" in master.columns else 0,
        "n_features_paths": int(master["features"].ne("—").sum()) if "features" in master.columns else 0,
    }

    # Colunas organizadas
    preferred = [
        "ID","audio","features","filename","title","artist","album","genre",
        "start","end","start_sec","end_sec",
        "valence_mean_static","arousal_mean_static",
        "valence_mean_dyn","arousal_mean_dyn",
        "valence_std_dyn","arousal_std_dyn",
        "valence_len_s","arousal_len_s","valence_npts","arousal_npts",
        "source_file"
    ]
    for p in preferred:
        if p not in master.columns:
            master[p] = pd.NA
    # reordena com preferidas na frente
    other_cols = [c for c in master.columns if c not in preferred]
    master = master[preferred + other_cols]

    # Avisos úteis
    missing_audio = master["audio"].eq("—").sum() if "audio" in master.columns else len(master)
    missing_VA = master[["valence_mean_static","arousal_mean_static","valence_mean_dyn","arousal_mean_dyn"]].isna().all(axis=1).sum()
    log(f"🔎 Cobertura: áudio ausente em {missing_audio} IDs | IDs sem qualquer VA em {missing_VA}")

    return master, report

# Need to call build_master with the global configuration variables
# master_df, summary = build_master(CFG) # Old call
# New call:
master_df, summary = build_master(BASE_DIR, META_FILES, STATIC_ANN_FILE, DYN_VAL_DIR, DYN_ARO_DIR)

log("Resumo:")
for k,v in summary.items():
    log(f"  - {k}: {v}")

# Visualização rápida (amostra)
display(master_df.head(20))

[02:28:41] ✅ IDs encontrados na árvore: 1802 (com audio/features)
[02:28:42] ✅ IDs únicos nos metadados: 753
[02:28:42] ℹ️  Aviso: 9 linhas sem duração de segmento (falta start/end válidos).
[02:28:42] 🔎 Cobertura: áudio ausente em 0 IDs | IDs sem qualquer VA em 1805
[02:28:42] Resumo:
[02:28:42]   - n_ids_tree: 1802
[02:28:42]   - n_ids_meta: 753
[02:28:42]   - n_audio_paths: 1805
[02:28:42]   - n_features_paths: 1805


/tmp/ipython-input-448882800.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  meta_all = pd.concat(frames, ignore_index=True, sort=False)


,ID,audio,features,filename,title,artist,album,genre,start,end,...,valence_mean_dyn,arousal_mean_dyn,valence_std_dyn,arousal_std_dyn,valence_len_s,arousal_len_s,valence_npts,arousal_npts,source_file,segment_duration_s
0,0,NaN,NaN,NaN,glittering raspberry dancefloors,QElectro,cruise control,Sex Party,do you really want crescentfresh tunes,NaN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,metadata_2014.csv,NaN
1,1,NaN,NaN,NaN,2 John,2 Peter,1 John,3 John,Book of Abraham,NaN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,metadata_2014.csv,NaN
2,2,audio/2.mp3,features/2.csv,2.mp3,Tonight A Lonely Century,The New Mystikal Troubadours,NaN,Blues,0.48,1.33,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,metadata_2013.csv,45.0
3,3,audio/3.mp3,features/3.csv,3.mp3,DD Groove,Kevin MacLeod,NaN,Blues,0.36,1.21,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,metadata_2013.csv,45.0
4,4,audio/4.mp3,features/4.csv,4.mp3,Slow Burn,Kevin MacLeod,NaN,Blues,1.29,2.14,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,metadata_2013.csv,45.0
5,5,audio/5.mp3,features/5.csv,5.mp3,Nothing Much,My Bubba & Mi,NaN,Blues,0.36,1.21,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,metadata_2013.csv,45.0
6,7,audio/7.mp3,features/7.csv,7.mp3,Hustle,Kevin MacLeod,NaN,Blues,0.2,1.05,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,metadata_2013.csv,63.0
7,8,audio/8.mp3,features/8.csv,8.mp3,Dirt Rhodes,Kevin MacLeod,NaN,Blues,0.12,0.57,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,metadata_2013.csv,45.0
8,10,audio/10.mp3,features/10.csv,10.mp3,Stormy Blues,Arne Bang Huseby,NaN,Blues,1.06,1.51,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,metadata_2013.csv,45.0
9,12,audio/12.mp3,features/12.csv,12.mp3,Rebel Blues,Sul Rebel,NaN,Blues,0.03,0.48,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,metadata_2013.csv,45.0
